In [8]:
from window_operate import *
from gaming_func import *
from notice_func import *
import sys,time,os
import threading

from ctypes import *
from ctypes.wintypes import *

def get_current_size(hwnd):
    try:
        f = ctypes.windll.dwmapi.DwmGetWindowAttribute
    except WindowsError:
        f = None
    if f:
        rect = ctypes.wintypes.RECT()
        DWMWA_EXTENDED_FRAME_BOUNDS = 9
        f(ctypes.wintypes.HWND(hwnd),
          ctypes.wintypes.DWORD(DWMWA_EXTENDED_FRAME_BOUNDS),
          ctypes.byref(rect),
          ctypes.sizeof(rect)
          )
        size = (rect.right - rect.left, rect.bottom - rect.top)        
        return size

hwnd = get_window_handle(title="咸鱼之王")
if hwnd == 0:
    print("游戏未启动，退出...")
    time.sleep(3)
    os._exit(1)
print_help()
print("Start...tap help for more details.")
w1,h1=get_current_size(hwnd)
rect = win32gui.GetWindowRect(hwnd)
x,y=rect[0],rect[1]
print(x,y)
w,h=rect[2] - x,rect[3] - y
print(w1,h1)
print(w,h)

-----------------
1.       显示窗口
2.       隐藏窗口
shoucai. 收离线奖励
tower.   咸将塔
help.    帮助
exit.    退出
-----------------
Start...tap help for more details.
978 138
905 1674
603 1116


In [4]:
from window_operate import *
from gaming_func import *
from notice_func import *
import sys,time,os
import threading

from ctypes import *
from ctypes.wintypes import *

def get_current_size(hwnd):
    try:
        f = ctypes.windll.dwmapi.DwmGetWindowAttribute
    except WindowsError:
        f = None
    if f:
        rect = ctypes.wintypes.RECT()
        DWMWA_EXTENDED_FRAME_BOUNDS = 9
        f(ctypes.wintypes.HWND(hwnd),
          ctypes.wintypes.DWORD(DWMWA_EXTENDED_FRAME_BOUNDS),
          ctypes.byref(rect),
          ctypes.sizeof(rect)
          )
        size = (rect.right - rect.left, rect.bottom - rect.top)        
        return size

hwnd = get_window_handle(title="咸鱼之王")
if hwnd == 0:
    print("游戏未启动，退出...")
    time.sleep(3)
    os._exit(1)
print_help()
print("Start...tap help for more details.")
w1,h1=get_current_size(hwnd)
rect = win32gui.GetWindowRect(hwnd)
x,y=rect[0],rect[1]
print(x,y)
w,h=rect[2] - x,rect[3] - y
print(w1,h1)
print(w,h)

-----------------
1.       显示窗口
2.       隐藏窗口
shoucai. 收离线奖励
tower.   咸将塔
help.    帮助
exit.    退出
-----------------
Start...tap help for more details.
24 9
675 1266
450 844


In [ ]:
x,y=rect[0],rect[1]
w,h=rect[2] - x,rect[3] - y